In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [13]:
results = pd.read_csv('results.csv')
results = results.query("simp_algorithm == ['OS','RDP','VC','BU']")
results.head(5)

,dataset,model,simp_algorithm,performance,"knee(x,y)",time
1,ProximalPhalanxOutlineCorrectTEST_normalized,logistic-regression_norm.pkl,BU,0.447152,"(0.04443037974683545, 0.0)",0.282440
2,ProximalPhalanxOutlineCorrectTEST_normalized,logistic-regression_norm.pkl,VC,0.872785,"(1.0, 1.0)",0.016343
3,ProximalPhalanxOutlineCorrectTEST_normalized,logistic-regression_norm.pkl,RDP,0.896519,"(0.13151898734177217, 1.0)",0.564308
6,ItalyPowerDemandTEST_normalized,logistic-regression_norm.pkl,BU,0.662132,"(0.14608695652173911, 0.4471744471744472)",0.051812
8,MoteStrainTEST_normalized,decision-tree_norm.pkl,BU,0.987952,"(1.0120481927710847, 1.0)",0.310772


In [14]:
print(len(results['dataset'].unique()))
print(len(results['model'].unique()))
print(len(results['simp_algorithm'].unique()))

22
4
4


In [15]:
os_alg = results.query('simp_algorithm == "OS"')["performance"].sum()
rdp_alg = results.query('simp_algorithm == "RDP"')["performance"].sum()
bu_alg = results.query('simp_algorithm == "BU"')["performance"].sum()
vc_alg = results.query('simp_algorithm == "VC"')["performance"].sum()
print("Sum of AUC per algorithm")
print("OS: ", os_alg)
print("RDP: ", rdp_alg)
print("BU: ", bu_alg)
print("VC: ", vc_alg)



Sum of AUC per algorithm
OS:  41.56891546931232
RDP:  52.241863209094525
BU:  55.29482684534684
VC:  58.78482881691798


In [16]:
os_alg = results.query('simp_algorithm == "OS"')["performance"].mean()
rdp_alg = results.query('simp_algorithm == "RDP"')["performance"].mean()
bu_alg = results.query('simp_algorithm == "BU"')["performance"].mean()
vc_alg = results.query('simp_algorithm == "VC"')["performance"].mean()
print("Mean of AUC per algorithm")
print("OS: ", os_alg)
print("RDP: ", rdp_alg)
print("BU: ", bu_alg)
print("VC: ", vc_alg)

Mean of AUC per algorithm
OS:  0.6113075804310636
RDP:  0.7463123315584932
BU:  0.7899260977906691
VC:  0.839783268813114


In [17]:
os_alg = results.query('simp_algorithm == "OS"')
os_alg = os_alg.groupby("model")["performance"].mean()
rdp_alg = results.query('simp_algorithm == "RDP"').groupby("model")["performance"].mean()
bu_alg = results.query('simp_algorithm == "BU"').groupby("model")["performance"].mean()
vc_alg = results.query('simp_algorithm == "VC"').groupby("model")["performance"].mean()
print("Mean of AUC per algorithm and model")
print("OS: ", os_alg)
print("RDP: ", rdp_alg)
print("BU: ", bu_alg)
print("VC: ", vc_alg)

Mean of AUC per algorithm and model
OS:  model
cnn_norm.pth                    0.634564
decision-tree_norm.pkl          0.581984
knn_norm.pkl                    0.585914
logistic-regression_norm.pkl    0.642313
Name: performance, dtype: float64
RDP:  model
cnn_norm.pth                    0.784299
decision-tree_norm.pkl          0.663398
knn_norm.pkl                    0.754213
logistic-regression_norm.pkl    0.784226
Name: performance, dtype: float64
BU:  model
cnn_norm.pth                    0.789248
decision-tree_norm.pkl          0.726481
knn_norm.pkl                    0.822051
logistic-regression_norm.pkl    0.821869
Name: performance, dtype: float64
VC:  model
cnn_norm.pth                    0.863470
decision-tree_norm.pkl          0.768100
knn_norm.pkl                    0.844616
logistic-regression_norm.pkl    0.881483
Name: performance, dtype: float64


## Only for CNN

In [18]:
results = results.query('model == "cnn_norm.pth"')

In [19]:
os_alg = results.query('simp_algorithm == "OS"')["performance"].sum()
rdp_alg = results.query('simp_algorithm == "RDP"')["performance"].sum()
bu_alg = results.query('simp_algorithm == "BU"')["performance"].sum()
vc_alg = results.query('simp_algorithm == "VC"')["performance"].sum()
print("Sum of AUC per algorithm")
print("OS: ", os_alg)
print("RDP: ", rdp_alg)
print("BU: ", bu_alg)
print("VC: ", vc_alg)

Sum of AUC per algorithm
OS:  10.153016911796167
RDP:  12.548780032805919
BU:  12.627969280984612
VC:  13.81551504202168


In [20]:
os_alg = results.query('simp_algorithm == "OS"')["performance"].mean()
rdp_alg = results.query('simp_algorithm == "RDP"')["performance"].mean()
bu_alg = results.query('simp_algorithm == "BU"')["performance"].mean()
vc_alg = results.query('simp_algorithm == "VC"')["performance"].mean()
print("Mean of AUC per algorithm")
print("OS: ", os_alg)
print("RDP: ", rdp_alg)
print("BU: ", bu_alg)
print("VC: ", vc_alg)

Mean of AUC per algorithm
OS:  0.6345635569872604
RDP:  0.7842987520503699
BU:  0.7892480800615382
VC:  0.863469690126355


In [21]:
datasets = results['dataset'].unique()

for dataset in datasets:
    dat_df = results.query(f'dataset == "{dataset}"')
    simp_algorithms = dat_df['simp_algorithm']
    #print(simp_algorithms)
    knees_df = dat_df["knee(x,y)"]
    #comb = zip(simp_algorithms, knees_df)
    knees = [[float(knee.replace("(", "").replace(")", "").split(",")[0]), float(knee.replace("(", "").replace(")", "").split(",")[1])] for knee in knees_df]

    x_values = [x for x, y in knees]
    y_values = [y for x, y in knees]

    min_x = min(x_values)
    min_x_index = x_values.index(min_x)
    y = y_values[min_x_index]

    y_max = max(y_values)
    if y == y_max:
        #sim_alg = simp_algorithms[min_x_index]
        print(f"Best model for {dataset} is {dat_df['model'].values[0]}, simp alg.  with AUC {y} at knee points {min_x, y}")
    

Best model for GunPointOldVersusYoungTEST_normalized is cnn_norm.pth, simp alg.  with AUC 1.0 at knee points (0.307248322147651, 1.0)
Best model for MiddlePhalanxTWTEST_normalized is cnn_norm.pth, simp alg.  with AUC 1.0 at knee points (0.1037974683544304, 1.0)
Best model for DistalPhalanxOutlineAgeGroupTEST_normalized is cnn_norm.pth, simp alg.  with AUC 1.0 at knee points (0.11468354430379747, 1.0)
Best model for ECG5000TEST_normalized is cnn_norm.pth, simp alg.  with AUC 1.0 at knee points (0.01043165467625899, 1.0)
Best model for MiddlePhalanxOutlineAgeGroupTEST_normalized is cnn_norm.pth, simp alg.  with AUC 1.0 at knee points (0.13708860759493674, 1.0)
Best model for ProximalPhalanxOutlineAgeGroupTEST_normalized is cnn_norm.pth, simp alg.  with AUC 1.0 at knee points (0.11797468354430382, 1.0)


In [22]:
#model resutls
#models_paths = [model_data for model_data in os.listdir() if ]

In [23]:
import numpy as np

electric_device = np.load("../data/ElectricDevices/ElectricDevices_TEST_normalized.npy")

labels = [arr[0] for arr in electric_device]
print(labels[:100])
print(set(labels[:100]))
#print(labels.count())


[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
{6.0}
